In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
!pip install -q sentence-transformers chromadb langchain-community langchain-core pillow

In [ ]:
from pathlib import Path
from typing import List
from PIL import Image

from sentence_transformers import SentenceTransformer
from langchain_core.embeddings import Embeddings
from langchain_community.vectorstores import Chroma

import json

In [ ]:
# # Đường dẫn tới file chunk hiện tại
# JSON_PATH = Path("/content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/chunks/qa_image_600.json")

# # 1) Đọc file
# data = json.loads(JSON_PATH.read_text(encoding="utf-8"))

# # 2) Duyệt từng câu hỏi và gán question_id
# for idx, q in enumerate(data, start=1):
#     # Nếu đã có "id" thì dùng luôn, không thì tạo mới
#     cur_id = q.get("id")
#     if not cur_id:
#         # Ví dụ: Q001, Q002, ..., Q600
#         cur_id = f"Q{idx:03d}"
#         q["id"] = cur_id

#     # Thêm (hoặc cập nhật) field question_id
#     q["question_id"] = cur_id

# # 3) Ghi đè lại file (hoặc ghi ra file mới)
# BACKUP_PATH = JSON_PATH.with_suffix(".backup.json")
# NEW_PATH = JSON_PATH  # nếu muốn ghi đè; hoặc .with_name("qa_chunks_600_with_qid.json")

# # Nếu muốn ghi đè file gốc thì:
# JSON_PATH.write_text(json.dumps(data, ensure_ascii=False, indent=2), encoding="utf-8")

# print("Đã thêm question_id cho", len(data), "câu hỏi.")
# print("File backup:", BACKUP_PATH)


Đã thêm question_id cho 599 câu hỏi.
File backup: /content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/chunks/qa_image_600.backup.json


In [ ]:
class ClipImageEmbeddings(Embeddings):
    """
    Embeddings cho ảnh: đầu vào là path ảnh (string),
    bên trong sẽ mở file ảnh và encode bằng CLIP.
    """

    def __init__(self, model_name: str = "clip-ViT-B-32"):
        self.model = SentenceTransformer(model_name)

    def _encode_one(self, image_path: str) -> List[float]:
        img = Image.open(image_path).convert("RGB")
        emb = self.model.encode(
            img,
            convert_to_numpy=True,
            normalize_embeddings=True,
        )
        return emb.tolist()

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """
        texts ở đây chính là list đường dẫn ảnh.
        LangChain sẽ gọi hàm này khi index.
        """
        return [self._encode_one(p) for p in texts]

    def embed_query(self, text: str) -> List[float]:
        """
        text = đường dẫn ảnh query.
        Dùng khi bạn similarity_search với 1 ảnh.
        """
        return self._encode_one(text)


In [ ]:
DATA_PATH = Path("/content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/chunks/qa_image_600.json")
data = json.loads(DATA_PATH.read_text(encoding="utf-8"))

image_paths: List[str] = []
image_metadatas: List[dict] = []
image_ids: List[str] = []

for idx, q in enumerate(data, start=1):
    # Lấy question_id (đã thêm ở bước trước)
    qid = q.get("question_id") or q.get("id") or f"Q{idx:03d}"

    refs = q.get("image_refs")

    # Bỏ qua câu không có ảnh
    if not refs:
        continue

    # Nếu image_refs là 1 string thì đưa về list
    if isinstance(refs, str):
        refs = [refs]

    for img_idx, img_path in enumerate(refs):
        if not img_path:
            continue

        # Tạo id duy nhất cho mỗi ảnh
        img_id = f"{qid}_img_{img_idx+1}"

        image_ids.append(img_id)
        image_paths.append(img_path)

        image_metadatas.append(
            {
                "id": img_id,
                "question_id": qid,
                "image_path": img_path,
                "source_doc": q.get("source_doc"),
                "modality": q.get("modality"),
            }
        )

print("Tổng số ảnh cần embed:", len(image_paths))


Tổng số ảnh cần embed: 317


In [ ]:
# Thư mục lưu vector của ảnh
CHROMA_IMG_DIR = Path("/content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/chroma_img_600")
CHROMA_IMG_DIR.mkdir(parents=True, exist_ok=True)

# Embedding model cho ảnh
img_embedder = ClipImageEmbeddings(model_name="clip-ViT-B-32")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

0_CLIPModel/pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

0_CLIPModel/model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
img_vectorstore = Chroma.from_texts(
    texts=image_paths,                    # mỗi phần tử: path ảnh
    embedding=img_embedder,               # model vision (CLIP)
    metadatas=image_metadatas,            # kèm question_id, source_doc,...
    ids=image_ids,                        # id duy nhất cho từng image
    persist_directory=str(CHROMA_IMG_DIR),
)

# Lưu xuống disk
img_vectorstore.persist()
print("✅ Đã embed và lưu IMAGE embeddings vào Chroma tại:", CHROMA_IMG_DIR)


✅ Đã embed và lưu IMAGE embeddings vào Chroma tại: /content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/chroma_img_600


/tmp/ipython-input-2856460570.py:10: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  img_vectorstore.persist()


In [ ]:
query_image_path = '/content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/data/images/qa600/img_question_227.png'

results = img_vectorstore.similarity_search(
    query_image_path,  # chính là "text" đầu vào, nhưng thực ra là path ảnh
    k=5,
)

for doc in results:
    print("  question_id:", doc.metadata.get("question_id"))
    print("  image_path:", doc.metadata.get("image_path"))
    print("  source_doc:", doc.metadata.get("source_doc"))
    print("-" * 40)

  question_id: Q227
  image_path: /content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/data/images/qa600/img_question_227.png
  source_doc: Bộ 600 câu hỏi dùng cho sát hạch lái xe cơ giới đường bộ.pdf
----------------------------------------
  question_id: Q288
  image_path: /content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/data/images/qa600/img_question_288.png
  source_doc: Bộ 600 câu hỏi dùng cho sát hạch lái xe cơ giới đường bộ.pdf
----------------------------------------
  question_id: Q287
  image_path: /content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/data/images/qa600/img_question_287.png
  source_doc: Bộ 600 câu hỏi dùng cho sát hạch lái xe cơ giới đường bộ.pdf
----------------------------------------
  question_id: Q229
  image_path: /content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/data/images/qa600/img_question_229.png
  source_doc: Bộ 600 câu hỏi dùng cho sát hạch lái xe cơ giới đường bộ.pdf
----------------------------------------
  qu